Este é um dos mais de 10 notebooks que utilizei nesta competição, sendo este focado exclusivamente na EDA do conjunto de teste. Aqui são realizadas etapas importantes, como seleção de variáveis, criação de novas colunas, limpeza de dados, preenchimento de valores nulos e tomadas de decisões que direcionaram o modelo final. 

In [1]:
import pandas as pd
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [2]:
train.shape

(188533, 13)

In [3]:
used = pd.read_csv('used_cars.csv')

In [4]:
used.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,Ford,Utility Police Interceptor Base,2013,"51,000 mi.",E85 Flex Fuel,300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capa...,6-Speed A/T,Black,Black,At least 1 accident or damage reported,Yes,"$10,300"
1,Hyundai,Palisade SEL,2021,"34,742 mi.",Gasoline,3.8L V6 24V GDI DOHC,8-Speed Automatic,Moonlight Cloud,Gray,At least 1 accident or damage reported,Yes,"$38,005"
2,Lexus,RX 350 RX 350,2022,"22,372 mi.",Gasoline,3.5 Liter DOHC,Automatic,Blue,Black,None reported,NaN,"$54,598"
3,INFINITI,Q50 Hybrid Sport,2015,"88,900 mi.",Hybrid,354.0HP 3.5L V6 Cylinder Engine Gas/Electric H...,7-Speed A/T,Black,Black,None reported,Yes,"$15,500"
4,Audi,Q3 45 S line Premium Plus,2021,"9,835 mi.",Gasoline,2.0L I4 16V GDI DOHC Turbo,8-Speed Automatic,Glacier White Metallic,Black,None reported,NaN,"$34,999"


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            188533 non-null  int64 
 1   brand         188533 non-null  object
 2   model         188533 non-null  object
 3   model_year    188533 non-null  int64 
 4   milage        188533 non-null  int64 
 5   fuel_type     183450 non-null  object
 6   engine        188533 non-null  object
 7   transmission  188533 non-null  object
 8   ext_col       188533 non-null  object
 9   int_col       188533 non-null  object
 10  accident      186081 non-null  object
 11  clean_title   167114 non-null  object
 12  price         188533 non-null  int64 
dtypes: int64(4), object(9)
memory usage: 18.7+ MB


In [6]:
used.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4009 entries, 0 to 4008
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   brand         4009 non-null   object
 1   model         4009 non-null   object
 2   model_year    4009 non-null   int64 
 3   milage        4009 non-null   object
 4   fuel_type     3839 non-null   object
 5   engine        4009 non-null   object
 6   transmission  4009 non-null   object
 7   ext_col       4009 non-null   object
 8   int_col       4009 non-null   object
 9   accident      3896 non-null   object
 10  clean_title   3413 non-null   object
 11  price         4009 non-null   object
dtypes: int64(1), object(11)
memory usage: 376.0+ KB


In [7]:
# Remover o símbolo do dólar e a vírgula
used['price'] = used['price'].str.replace('$', '').str.replace(',', '')

# Converter para numérico
used['price'] = pd.to_numeric(used['price'])

In [8]:
#Remover a vírgula, o espaço e a string 'mi.'
used['milage'] = used['milage'].str.replace(',', '').str.replace(' mi.', '')

# Converter para int64
used['milage'] = used['milage'].astype('int64')

In [9]:
train = train.drop("id",axis=1)

In [10]:
used.shape

(4009, 12)

In [11]:
# Concatenar os dois DataFrames
df = pd.concat([train, used], ignore_index=True)

df.shape

(192542, 12)

In [12]:
df.tail()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
192537,Bentley,Continental GT Speed,2023,714,Gasoline,6.0L W12 48V PDI DOHC Twin Turbo,8-Speed Automatic with Auto-Shift,C / C,Hotspur,None reported,Yes,349950
192538,Audi,S4 3.0T Premium Plus,2022,10900,Gasoline,349.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,53900
192539,Porsche,Taycan,2022,2116,NaN,Electric,Automatic,Black,Black,None reported,NaN,90998
192540,Ford,F-150 Raptor,2020,33000,Gasoline,450.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,A/T,Blue,Black,None reported,Yes,62999
192541,BMW,X3 xDrive30i,2020,43000,Gasoline,248.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,A/T,Gray,Brown,At least 1 accident or damage reported,Yes,40000


In [13]:
df['fuel_type'].value_counts()

fuel_type
Gasoline          169249
Hybrid              7026
E85 Flex Fuel       5545
Diesel              4071
–                    826
Plug-In Hybrid       555
not supported         17
Name: count, dtype: int64

In [14]:
df['engine'].value_counts()

engine
355.0HP 5.3L 8 Cylinder Engine Gasoline Fuel                        3510
240.0HP 2.0L 4 Cylinder Engine Gasoline Fuel                        2944
420.0HP 6.2L 8 Cylinder Engine Gasoline Fuel                        2888
2.0L I4 16V GDI DOHC Turbo                                          2732
375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel                       2471
                                                                    ... 
6.6L V-12 gasoline direct injection, DOHC, variable valve contro       1
HEMI 5.7L V-8 variable valve control, regular unleaded, engine w       1
1.6L I-4 gasoline direct injection, DOHC, variable valve control       1
AMG 3L I-6 gasoline direct injection, DOHC, variable valve contr       1
3.3L V-6 gasoline direct injection, DOHC, D-CVVT variable valve        1
Name: count, Length: 1146, dtype: int64

## Criando a variavel 'LITROS'

In [15]:
# Usar regex para extrair os números antes de 'L' ou 'Liter'
df['litros'] = df['engine'].str.extract(r'(\d+\.\d+)(?:L| Liter)', expand=False)

# Converter para float
df['litros'] = df['litros'].astype(float)

In [16]:
df['litros'].isnull().sum()

7008

In [87]:
# Ver algumas linhas onde a coluna 'litros' é NaN
linhas_nan = df[df['cambio'].isna()]

In [90]:
linhas_nan.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,litros,cylinder,cavalo,km,cambio


In [32]:
df.isnull().sum()

brand               0
model               0
model_year          0
milage              0
fuel_type         182
engine              0
transmission        0
ext_col             0
int_col             0
accident         2565
clean_title     22015
price               0
litros           7008
cylinder        38800
cavalo          34067
dtype: int64

In [20]:
train.isnull().sum()

brand               0
model               0
model_year          0
milage              0
fuel_type        5083
engine              0
transmission        0
ext_col             0
int_col             0
accident         2452
clean_title     21419
price               0
dtype: int64

In [21]:
# Função para atualizar fuel_type com base na coluna engine
def update_fuel_type(row):
    engine = row['engine']
    if 'Electric' in engine:
        return 'Hybrid'
    elif 'Gasoline' in engine:
        return 'Gasoline'
    elif 'Flex' in engine:
        return 'E85 Flex Fuel'
    elif 'Diesel' in engine:
        return 'Diesel'
    return row['fuel_type']  # Retorna o valor atual se nenhuma condição for satisfeita

# Aplicar a função na coluna engine para atualizar a coluna fuel_type
df['fuel_type'] = df.apply(update_fuel_type, axis=1)

df['fuel_type'].value_counts()

fuel_type
Gasoline          168149
Hybrid             13424
E85 Flex Fuel       5641
Diesel              4303
–                    819
Plug-In Hybrid        24
Name: count, dtype: int64

In [22]:
# Substituir 'Plug-In Hybrid' por 'Hybrid'
df['fuel_type'] = df['fuel_type'].replace({'Plug-In Hybrid': 'Hybrid'})

# Verificar o resultado
print(df['fuel_type'].value_counts())

fuel_type
Gasoline         168149
Hybrid            13448
E85 Flex Fuel      5641
Diesel             4303
–                   819
Name: count, dtype: int64


In [23]:
# Substituir 'Plug-In Hybrid' por 'Hybrid'
df['fuel_type'] = df['fuel_type'].replace({'E85 Flex Fuel': 'Flex'})

# Verificar o resultado
print(df['fuel_type'].value_counts())

fuel_type
Gasoline    168149
Hybrid       13448
Flex          5641
Diesel        4303
–              819
Name: count, dtype: int64


In [24]:
fuel_preenchido = df.copy()

In [25]:

# Salvar DataFrame como arquivo CSV
#fuel_preenchido.to_csv('fuel_preenchido.csv', index=False)

In [26]:
fuel_preenchido.sample(10)

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,litros
89410,Jeep,Gladiator Overland,2020,7388,Gasoline,3.6L V6 24V MPFI DOHC,8-Speed Automatic,Firecracker Red Clearcoat,Black,NaN,NaN,44950,3.6
121097,Jeep,Gladiator Rubicon,2023,7000,Gasoline,285.0HP 3.6L V6 Cylinder Engine Gasoline Fuel,A/T,Green,Black,None reported,Yes,50000,3.6
94249,Mercedes-Benz,G-Class G 550 4MATIC,2019,32376,Gasoline,416.0HP 4.0L 8 Cylinder Engine Gasoline Fuel,A/T,Gray,Black,None reported,Yes,60500,4.0
5083,Chevrolet,Corvette Stingray w/2LT,2022,20121,Gasoline,490.0HP 6.2L 8 Cylinder Engine Gasoline Fuel,A/T,Black,Black,None reported,Yes,94999,6.2
17866,Jaguar,F-TYPE R,2020,28470,Gasoline,550.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,8-Speed A/T,White,White,None reported,Yes,47500,5.0
92882,Ford,Sprinter 2500 High Roof,2018,45000,Diesel,188.0HP 3.0L V6 Cylinder Engine Diesel Fuel,A/T,White,Black,At least 1 accident or damage reported,Yes,35200,3.0
95255,Genesis,G90 5.0 Ultimate,2014,82975,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,8-Speed A/T,White,Black,At least 1 accident or damage reported,Yes,42850,5.0
102948,BMW,228 Gran Coupe i xDrive,2022,2800,Gasoline,255.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,M/T,Blue,Gray,None reported,Yes,21999,2.0
99887,GMC,Acadia SLT-2,2013,93500,Gasoline,288.0HP 3.6L V6 Cylinder Engine Gasoline Fuel,A/T,Brown,Beige,None reported,Yes,17999,3.6
38626,Toyota,Camry Solara SLE V6,2007,121800,Gasoline,210.0HP 3.3L V6 Cylinder Engine Gasoline Fuel,A/T,White,Black,None reported,Yes,9499,3.3


In [27]:
import re
import numpy as np

# Função para extrair o número de cilindros
def extract_cylinder(engine_text):
    match = re.search(r'(\d+)\sCylinder', engine_text)
    if match:
        return int(match.group(1))
    else:
        return np.nan

# Criar a nova coluna 'cylinder' com base na coluna 'engine'
df['cylinder'] = df['engine'].apply(extract_cylinder)

df.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,litros,cylinder
0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200,1.6,4.0
1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999,3.9,8.0
2,Chevrolet,Silverado 2500 LT,2002,136731,Flex,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900,5.3,8.0
3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000,5.0,8.0
4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500,2.0,4.0


In [75]:
df.isnull().sum()

brand               0
model               0
model_year          0
milage              0
fuel_type         182
engine              0
transmission        0
ext_col             0
int_col             0
accident         2565
clean_title     22015
price               0
litros           7008
cylinder        15626
cavalo          34067
km                  0
dtype: int64

In [29]:
# Função para extrair o número antes de 'HP'
def extract_horsepower(engine_text):
    match = re.search(r'(\d+\.?\d*)HP', engine_text)
    if match:
        return float(match.group(1))
    else:
        return np.nan

# Criar a nova coluna 'cavalo' com base na coluna 'engine'
df['cavalo'] = df['engine'].apply(extract_horsepower)

In [30]:
df.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,litros,cylinder,cavalo
0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200,1.6,4.0,172.0
1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999,3.9,8.0,252.0
2,Chevrolet,Silverado 2500 LT,2002,136731,Flex,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900,5.3,8.0,320.0
3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000,5.0,8.0,420.0
4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500,2.0,4.0,208.0


In [31]:
# Função para extrair o número de cilindros somente quando há 'I'
def extract_cylinders_from_I(engine_text):
    # Captura número de cilindros antes do 'I' (maiúsculo ou minúsculo)
    match = re.search(r'(\d+)\s*I', engine_text)
    if match:
        return int(match.group(1))
    else:
        return np.nan

# Atualizar a coluna 'cylinder' onde o valor é NaN
df['cylinder'] = df.apply(lambda row: extract_cylinders_from_I(row['engine']) if pd.isna(row['cylinder']) else row['cylinder'], axis=1)

df.isnull().sum()

brand               0
model               0
model_year          0
milage              0
fuel_type         182
engine              0
transmission        0
ext_col             0
int_col             0
accident         2565
clean_title     22015
price               0
litros           7008
cylinder        38800
cavalo          34067
dtype: int64

In [50]:
# Função para extrair o número logo após o 'I' maiúsculo
def extract_cylinders_after_I(engine_text):
    # Regex para capturar número após o 'I' maiúsculo
    match = re.search(r'I(\d+)', engine_text)
    if match:
        return int(match.group(1))
    return np.nan

# Atualizar a coluna 'cylinder' com base na coluna 'engine'
df['cylinder'] = df.apply(lambda row: extract_cylinders_after_I(row['engine']) if pd.isna(row['cylinder']) else row['cylinder'], axis=1)



df.isnull().sum()

brand               0
model               0
model_year          0
milage              0
fuel_type         182
engine              0
transmission        0
ext_col             0
int_col             0
accident         2565
clean_title     22015
price               0
litros           7008
cylinder        32041
cavalo          34067
dtype: int64

In [69]:
# Função para extrair o número logo após o 'I' maiúsculo
def extract_cylinders_after_I(engine_text):
    # Regex para capturar número após o 'I' maiúsculo
    match = re.search(r'V(\d+)', engine_text)
    if match:
        return int(match.group(1))
    return np.nan

# Atualizar a coluna 'cylinder' com base na coluna 'engine'
df['cylinder'] = df.apply(lambda row: extract_cylinders_after_I(row['engine']) if pd.isna(row['cylinder']) else row['cylinder'], axis=1)



df.isnull().sum()

brand               0
model               0
model_year          0
milage              0
fuel_type         182
engine              0
transmission        0
ext_col             0
int_col             0
accident         2565
clean_title     22015
price               0
litros           7008
cylinder        15626
cavalo          34067
km                 10
dtype: int64

In [54]:
df.sample(10)

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,litros,cylinder,cavalo
186075,Ford,Expedition EL XLT,2019,85149,Gasoline,3.5L V6 24V PDI DOHC Twin Turbo,10-Speed Automatic,White,Ebony,None reported,NaN,32769,3.5,6.0,NaN
72229,GMC,Sierra 1500 AT4,2021,56247,Gasoline,5.3L V8 16V GDI OHV,10-Speed Automatic,Black,Jet Black,None reported,NaN,27633,5.3,8.0,NaN
158199,Honda,Civic EX,2016,90000,Gasoline,159.0HP 2.7L 4 Cylinder Engine Gasoline Fuel,A/T,White,Gray,None reported,Yes,19900,2.7,4.0,159.0
95392,Toyota,Avalon Hybrid XLE Premium,2017,117878,Hybrid,200.0HP 2.5L 4 Cylinder Engine Gas/Electric Hy...,CVT Transmission,Silver,Gray,None reported,Yes,28700,2.5,4.0,200.0
143025,Toyota,Sequoia Limited,2022,94100,Gasoline,381.0HP 5.7L 8 Cylinder Engine Gasoline Fuel,A/T,Green,Boulder,At least 1 accident or damage reported,Yes,81900,5.7,8.0,381.0
177210,Ford,F-150 Raptor,2015,89988,Gasoline,450.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,A/T,Black,Black,At least 1 accident or damage reported,Yes,26500,3.5,6.0,450.0
47582,RAM,2500 Tradesman,2018,53900,Gasoline,410.0HP 6.4L 8 Cylinder Engine Gasoline Fuel,8-Speed A/T,Black,Gray,None reported,Yes,35700,6.4,8.0,410.0
65288,BMW,ALPINA B7 ALPINA B7 xDrive,2017,51983,Gasoline,4.4 Liter Twin Turbo,Automatic,Black,Black,None reported,Yes,32769,4.4,NaN,NaN
36296,Chevrolet,Corvette Z06,2016,42000,Gasoline,650.0HP 6.2L 8 Cylinder Engine Gasoline Fuel,6-Speed M/T,Black,Black,None reported,Yes,52500,6.2,8.0,650.0
57697,Toyota,Tacoma PreRunner,2018,20800,Gasoline,236.0HP 4.0L V6 Cylinder Engine Gasoline Fuel,A/T,Black,Gray,At least 1 accident or damage reported,Yes,36500,4.0,6.0,236.0


In [55]:
meio_preenchido = df.copy()


# Salvar DataFrame como arquivo CSV
meio_preenchido.to_csv('meio_preenchido.csv', index=False)

In [58]:
df = df.drop('km',axis=1)

In [72]:
# Definir os intervalos de agrupamento (por exemplo, a cada 10.000 km)
bins = list(range(0, 500001, 10000))  # Ajuste os limites conforme necessário
labels = [str(x) for x in bins[:-1]]  # Labels apenas com o número inicial do intervalo

# Criar a nova coluna com base nos intervalos definidos
df['km'] = pd.cut(df['milage'], bins=bins, labels=labels, right=False).astype(float)

In [76]:
df.sample(8)

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,litros,cylinder,cavalo,km
28365,Mercedes-Benz,E-Class E 350,2021,16000,Gasoline,255.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,9-Speed A/T,White,Beige,None reported,Yes,29998,2.0,4.0,255.0,10000.0
86867,BMW,M3 Base,2008,130000,Gasoline,333.0HP 3.2L Straight 6 Cylinder Engine Gasoli...,6-Speed M/T,Silver,Black,None reported,Yes,3000,3.2,6.0,333.0,130000.0
73224,Ford,F-150 XLT,2018,54300,Gasoline,375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,White,Black,None reported,Yes,44900,3.5,6.0,375.0,50000.0
55308,Lexus,GX 460 Premium,2021,10339,Gasoline,301.0HP 4.6L 8 Cylinder Engine Gasoline Fuel,A/T,Black,Black,None reported,Yes,32455,4.6,8.0,301.0,10000.0
78210,Land,Rover Range Rover Velar P380 S,2019,47600,Gasoline,380.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,51900,3.0,6.0,380.0,40000.0
185589,MINI,Cooper S Clubman Base,2009,79000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,6-Speed M/T,White,Black,At least 1 accident or damage reported,Yes,4900,1.6,4.0,172.0,70000.0
81799,Kia,Telluride SX,2020,39000,Gasoline,291.0HP 3.8L V6 Cylinder Engine Gasoline Fuel,8-Speed A/T,Silver,Gray,None reported,Yes,50900,3.8,6.0,291.0,30000.0
189591,Land,Rover Range Rover P400 SE SWB,2023,11700,Hybrid,395.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,Transmission w/Dual Shift Mode,Gray,White,None reported,Yes,133500,3.0,6.0,395.0,10000.0


In [74]:
# Substituir 0.0 por 1000.0
df['km'].replace(0.0, 1000.0, inplace=True)

In [77]:
df['transmission'].value_counts()

transmission
A/T                                                                 50941
8-Speed A/T                                                         21051
Transmission w/Dual Shift Mode                                      19653
6-Speed A/T                                                         18406
6-Speed M/T                                                         12246
                                                                    ...  
Automatic, 8-Spd PDK Dual-Clutch                                        1
Automatic, 8-Spd M STEPTRONIC w/Drivelogic, Sport & Manual Modes        1
Automatic, 8-Spd Dual-Clutch                                            1
Automatic, 10-Spd                                                       1
Automatic, 7-Spd S tronic Dual-Clutch                                   1
Name: count, Length: 62, dtype: int64

In [101]:
# Função para determinar o tipo de câmbio
def classify_cambio(transmission_text):
    if 'Automatic' in transmission_text or 'A/T' in transmission_text or 'Dual Shift Mode' in transmission_text or 'CVT' in transmission_text or "Overdrive Switch" in transmission_text:
        return 'Auto'
    elif 'M/T' in transmission_text or 'manual' in transmission_text or 'Manual' in transmission_text or 'Mt' in transmission_text:
        return 'Manual'
    else:
        return 'Unknown'

# Criar a nova coluna 'cambio'
df['cambio'] = df['transmission'].apply(classify_cambio)


In [102]:
df['cambio'].value_counts()

cambio
Auto       175334
Manual      16789
Unknown       419
Name: count, dtype: int64

In [104]:
# Filtrar as linhas onde a coluna 'cambio' é 'Unknown'
unknown_cambio_df = df[df['cambio'] == 'Unknown']

In [105]:
# Exibir as linhas com 'Unknown' em 'cambio'
unknown_cambio_df.sample(10)

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,litros,cylinder,cavalo,km,cambio
188232,Acura,MDX w/Technology Package,2023,3521,Gasoline,3.5 Liter SOHC,F,Majestic Black Pearl,Parchment,None reported,NaN,78900,3.5,NaN,NaN,1000.0,Unknown
65863,Acura,ILX Premium Package,2021,24651,Gasoline,2.4L I4 16V GDI DOHC,2,Gray,Ebony,None reported,NaN,31853,2.4,4.0,NaN,20000.0,Unknown
161062,Porsche,911 Carrera S,2021,10071,Gasoline,3.0 Liter,–,White,Black,None reported,NaN,32913,3.0,NaN,NaN,10000.0,Unknown
162871,BMW,M5 Base,2016,816,Hybrid,Dual AC Electric Motors,Single-Speed Fixed Gear,Alpine White,Anthracite,None reported,Yes,41999,NaN,NaN,NaN,1000.0,Unknown
21608,Acura,MDX 3.5L w/Technology Package,2019,90605,Gasoline,3.5 Liter,F,White,Parchment,None reported,NaN,35999,3.5,NaN,NaN,90000.0,Unknown
118586,Hummer,H2 Base,2005,110000,Gasoline,325.0HP 6.0L 8 Cylinder Engine Gasoline Fuel,2,White,Gray,None reported,Yes,11000,6.0,8.0,325.0,110000.0,Unknown
106696,Ford,Edge SEL,2019,35340,Gasoline,2.0 Liter TFSI,2,Silver,Ebony,None reported,NaN,27698,2.0,NaN,NaN,30000.0,Unknown
56630,Lamborghini,Aventador SVJ Base,2018,6987,Gasoline,6.5L V12 48V MPFI DOHC,7-Speed,–,Nero Ade,None reported,Yes,39999,6.5,12.0,NaN,1000.0,Unknown
84170,Lexus,NX 350 F SPORT Handling,2023,820,Gasoline,2.4 Liter,8-SPEED AT,Black,Black,None reported,NaN,53900,2.4,NaN,NaN,1000.0,Unknown
127398,Nissan,Maxima SV,2022,30554,Gasoline,3.5L V6 24V GDI DOHC Twin Turbo,2,Magnetic Gray Clearcoat,Black,None reported,NaN,29614,3.5,6.0,NaN,30000.0,Unknown


In [106]:
# Substituir 'Unknown' por 'Manual' na coluna 'cambio'
df['cambio'] = df['cambio'].replace('Unknown', 'Manual')

# Verificar a contagem das categorias após a substituição
df['cambio'].value_counts()

cambio
Auto      175334
Manual     17208
Name: count, dtype: int64

In [107]:
df.isnull().sum()

brand               0
model               0
model_year          0
milage              0
fuel_type         182
engine              0
transmission        0
ext_col             0
int_col             0
accident         2565
clean_title     22015
price               0
litros           7008
cylinder        15626
cavalo          34067
km                  0
cambio              0
dtype: int64

In [110]:
df = df.drop('clean_title',axis=1)

In [144]:
df['interna'].value_counts()

interna
Black                        114285
Beige                         25033
Gray                          22072
Outros                         6321
Red                            6113
Brown                          5967
White                          5004
Ebony                          1877
Orange                         1024
Blue                            888
Silver                          445
Nero Ade                        399
Gold                            377
Graphite                        366
Charcoal                        222
Green                           202
Parchment                       174
Medium Stone                    172
Sandstone                       162
Tan                             153
Shara Beige                     129
Hotspur Hide                    104
Espresso                        100
Canberra Beige                   90
Light Slate                      72
Ice                              71
Walnut                           70
Silk Beige/Espresso 

In [132]:
df['int_col'].value_counts()

int_col
Black           109699
Beige            25033
Gray             21676
Brown             5967
Red               5271
                 ...  
Nougat Brown         8
Classic Red          8
Cobalt Blue          7
WHITE                7
ORANGE               5
Name: count, Length: 156, dtype: int64

In [140]:

# Supondo que 'df' seja seu DataFrame e 'ext_col' seja a coluna com as cores

# Atualizar a função para categorizar cores
def categorize_color(color):
    if pd.notna(color):
        color_lower = color.lower()
        if 'graphite' in color_lower:
            return 'Graphite'
        if 'orange' in color_lower:
            return 'Orange'
        if 'white' in color_lower:
            return 'White'
        if 'metallic' in color_lower:
            return 'Metallic'
        elif 'blue' in color_lower:
            return 'Blue'
        elif 'red' in color_lower:
            return 'Red'
        elif 'black' in color_lower:
            return 'Black'
        elif 'white' in color_lower:
            return 'White'
        elif 'gray' in color_lower or 'grey' in color_lower:
            return 'Gray'
        elif 'silver' in color_lower:
            return 'Silver'
        elif color in rare_colors:
            return 'Outros'
    return color


In [141]:
# Contar a frequência de cada cor
color_counts = df['int_col'].value_counts()

# Identificar as cores que aparecem menos de 20 vezes
rare_colors = color_counts[color_counts < 50].index

# Aplicar a função para criar a nova coluna 'externa'
df['interna'] = df['int_col'].apply(categorize_color)


# Verificar as primeiras linhas para garantir que a coluna foi criada corretamente
df[['int_col', 'interna']].head()

,int_col,interna
0,Gray,Gray
1,Beige,Beige
2,Gray,Gray
3,Black,Black
4,Beige,Beige


In [145]:
df.sample(8)

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,price,litros,cylinder,cavalo,km,cambio,externa,interna
96789,Land,Rover Range Rover Evoque HSE Dynamic,2016,79000,Gasoline,240.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,9-Speed A/T,Red,Beige,None reported,23500,2.0,4.0,240.0,70000.0,Auto,Red,Beige
170047,Mercedes-Benz,G-Class G 550 4MATIC,2018,41500,Gasoline,416.0HP 4.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,White,Black,None reported,67000,4.0,8.0,416.0,40000.0,Auto,White,Black
117160,Toyota,Highlander XLE,2020,18000,Gasoline,295.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,A/T,Black,Black,At least 1 accident or damage reported,30000,3.5,6.0,295.0,10000.0,Auto,Black,Black
147869,Lincoln,LS V8,2015,60000,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,7-Speed A/T,White,Black,None reported,28000,3.9,8.0,252.0,60000.0,Auto,White,Black
163942,Hyundai,Genesis 3.8,2016,80300,Gasoline,311.0HP 3.8L V6 Cylinder Engine Gasoline Fuel,8-Speed A/T,Gray,Black,None reported,28500,3.8,6.0,311.0,80000.0,Auto,Gray,Black
45504,Lincoln,Navigator Reserve,2019,64855,Gasoline,450.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,Black,Beige,None reported,29900,3.5,6.0,450.0,60000.0,Auto,Black,Beige
82612,Mercedes-Benz,E-Class E 350,2012,165520,Gasoline,302.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,8-Speed A/T,Black,Beige,At least 1 accident or damage reported,9000,3.5,6.0,302.0,160000.0,Auto,Black,Beige
82765,GMC,Sierra 2500 AT4,2022,2550,Gasoline,401.0HP 6.6L 8 Cylinder Engine Gasoline Fuel,10-Speed A/T,Gray,Black,None reported,52000,6.6,8.0,401.0,1000.0,Auto,Gray,Black


In [143]:

# Substituir '-' por 'Outros'
df['interna'] = df['interna'].replace('–', 'Outros')

# Verificar as primeiras linhas para garantir que a coluna foi atualizada corretamente
df[['interna']].head()

,interna
0,Gray
1,Beige
2,Gray
3,Black
4,Beige


In [146]:
quasetudo = df.copy()

In [147]:
# Salvar DataFrame como arquivo CSV
quasetudo.to_csv('quasetudo.csv', index=False)